In [1]:
include("../main.jl")
using PyCall;
using Revise;
using Gen;
using CairoMakie
using Serialization
sepsis_gym = pyimport("custom_sepsis");

In [ ]:
models = Dict(
    :SimplePPL => [MCMCModel(:SimplePPL, choicemap(), [], []) for i in 1:10],
    :Softmax => [MCMCModel(:Softmax, choicemap(), [], []) for i in 1:10]
)
ts_runs = Dict(
    :SimplePPL => [PPTSRun("history-$(label_dict[model.type])-$i", model) for (i, model) in enumerate(models[:SimplePPL])],
    :Softmax => [PPTSRun("history-$(label_dict[model.type])-$i", model) for (i, model) in enumerate(models[:Softmax])],
)

indices = [2^i for i in [0,1,2,3,4,5,6,8,10,12,14,17,20,26]]
history_size = maximum(indices)

pols = [sepsis_gym.random_policy() for _ in 1:history_size]
episodes = [sepsis_gym.run_episode(pols[i]) for i in 1:history_size]

In [ ]:
for i in indices
    for type in [:SimplePPL, :Softmax]
        for history_run in history_runs[type]
            iterations = i == 1 ? 200 : 20
            run_history_mcmc!(history_run, i, iterations)
            policy = history_run.policies[i]
            update_model_history!(history_run.model, i, policy)
            save_ts_jld(history_run)
            println("Finished episode ", history_run.name, " ", i)
        end
    end
end

In [6]:
maximum(keys(histories[1].models))

2048

In [ ]:
for i in 2049:100000000
    for history in histories
        if log2(i) % 1 == 0 
            update_model!(history.model, i)
            run_history_mcmc!(history, i, 100)
            save_history_jld(history)
            println("Finished episode ", history.model.type, " ", i)
        end
    end
end